In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install IMDbPY
!pip install spotipy
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master
import pandas as pd
import html
import imdb
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from imdb import IMDb
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from random import randint
import requests
from polyglot.text import Text

client_id = '330e84ce3b4e4cceacb8b932b1c11bb6'
client_secret = '9b51dbb9f20e48a1966359108fe47848'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
data = pd.read_pickle('/content/drive/My Drive/mlschool/videos.p')
data.head()

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-9oe2wyvq
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-9oe2wyvq
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=70644 sha256=ade57300116cd343b8050fe9f3191986df69e9d9b8229686258d1a53c950ea40
  Stored in directory: /tmp/pip-ephem-wheel-cache-3s1985d3/wheels/42/d9/73/345c7ae8554299ff8b31635d64eb8455fd591385fa734cdbef
Successfully built polyglot
  Found existing installation: polyglot 16.7.4
    Uninstalling polyglot-16.7.4:
      Successfully uninstalled polyglot-16.7.4


,fromId,vid
0,100001643,"[Zivert - Beverly Hills | mood «Красотка», Sta..."
1,1000067,"[Игорь Растеряев. Плясовая., Спорт, спорт, спо..."
2,1000117,[Харламов и Батрутдинов&quot;На газовых перего...
3,100012291,"[Сжигаем жир, Вкусный и сытный завтрак, No nam..."
4,10001414,"[Avicii - Wake Me Up (Official Video), В джазе..."


In [3]:
data_new = pd.DataFrame(columns=['music', 'films', 'other', 'genres_of_music', 'genres_of_films'])
data_new.head()

,music,films,other,genres_of_music,genres_of_films


In [13]:
data_of_labels = pd.read_csv('/content/drive/My Drive/mlschool/music_films_labels.csv')
data_of_labels.head()

,fromId,"Фильм-1,Музыка-2",vid
0,100402039,NaN,Приколы студентов
1,100402039,NaN,приколы с зубной пастой
2,100402039,NaN,"Прикол, всё любят делать за деньги&#33;=)"
3,100402039,NaN,Кот матерится)
4,100402039,NaN,Кот пердун


In [ ]:
def get_continuous_chunks(text):
     chunked = ne_chunk(pos_tag(word_tokenize(text)))
     continuous_chunk = []
     current_chunk = []
     for i in chunked:
             if type(i) == Tree:
                     current_chunk.append(" ".join([token for token, pos in i.leaves()]))
             elif current_chunk:
                     named_entity = " ".join(current_chunk)
                     if named_entity not in continuous_chunk:
                             continuous_chunk.append(named_entity)
                             current_chunk = []
             else:
                     continue
     return continuous_chunk

def is_it_cinema(name):
    movies = ia.search_movie(name)
    if len(movies) > 0:
      movie = movies[0]
      ia.update(movie, info=['main'])
      try:
        return 1, name, movie['genres']
      except KeyError:
        return 1, name, None
    return 0, None, None


def is_it_music(name):
    try:
      result = sp.search(name)
      if len(result['tracks']['items']) > 0:
        track = result['tracks']['items'][0]
        if track != None:
          artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
          return 2, name, artist["genres"]
        else:
          return 2, name, None
      else:
        return 0, None, None
    except requests.ReadTimeout:
      return 0, None, None

In [ ]:
ia = IMDb()
for i in range(1, len(data)):
    films = []
    music = []
    other = []
    genres_of_music = []
    genres_of_films = []
    if len(data['vid'][i]) > 20:
      count = 20
    else:
      count = len(data['vid'][i])
    for j in range(count):
      number = randint(0, len(data['vid'][i]) - 1)
      line = data['vid'][i][number]
      line = html.unescape(line)
      names = get_continuous_chunks(line)
      k = -1
      if names != None:
        while (k + 1< len(names)):
          k += 1
          flag, name_films, genres_films = is_it_cinema(names[k])
          if flag == 1:
            films.append(name_films)
            genres_of_films.append(genres_films)
            break
          else:
            flag, name_music, genres_music = is_it_music(names[k])
            if flag == 2:
              music.append(name_music)
              genres_of_music.append(genres_music)
              break
            else:
              other.append(line)

    genres_of_music = [i for j in genres_of_music for i in j]

    genres_of_films = [i for j in genres_of_films for i in j]

    data_new.loc[i] = [music, films, other, genres_of_music, genres_of_films]



